In [ ]:
# Rok Cerne
# Rokcernejr1@gmail.com
# ClusterQuest - Unveiling Cohorts in Social Networks

# Step 1: Install necessary libraries
!pip install pandas networkx matplotlib seaborn scikit-learn node2vec

# Step 2: Import necessary libraries
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, SpectralClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# Step 3: Load and prepare the data
from google.colab import files
uploaded = files.upload()  # Upload facebook_combined.txt

data = pd.read_csv('facebook_combined.txt', sep=' ', header=None, names=['node1', 'node2'])

# Step 4: Create a graph from the data
G = nx.from_pandas_edgelist(data, 'node1', 'node2')

# Step 5: Remove isolated nodes
G.remove_nodes_from(list(nx.isolates(G)))

# Printing the number of nodes and edges after removing isolated nodes
print(f"Number of nodes after removing isolated nodes: {G.number_of_nodes()}")
print(f"Number of edges after removing isolated nodes: {G.number_of_edges()}")

# Explanation for Step 5:
# This section of code creates the graph structure and removes isolated nodes.
# Isolated nodes do not contribute to the graph's connected components and can distort clustering results.
# By removing them, we ensure the analysis focuses on meaningful connections within the graph.

# Step 6: Generate node embeddings using Node2Vec
from node2vec import Node2Vec
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Step 7: Create embeddings DataFrame
embeddings = pd.DataFrame([model.wv.get_vector(str(n)) for n in G.nodes()], index=G.nodes())

# Step 8: Scale the embeddings
scaler = StandardScaler()
X_scaled = scaler.fit_transform(embeddings)

# Explanation for Steps 6-8:
# Node2Vec generates embeddings that capture the graph's structural and relational properties.
# These embeddings are transformed into a feature space that machine learning algorithms can utilize.
# Scaling the embeddings ensures that each feature contributes equally to the clustering process.

# Step 9: Define custom grid search function with additional metrics
def custom_grid_search(estimator, param_grid, X):
    best_score = -1
    best_params = None
    best_db_score = float('inf')
    best_ch_score = -1
    for params in param_grid:
        estimator.set_params(**params)
        labels = estimator.fit_predict(X)
        if len(set(labels)) > 1:  # Avoid single cluster
            sil_score = silhouette_score(X, labels)
            db_score = davies_bouldin_score(X, labels)
            ch_score = calinski_harabasz_score(X, labels)
            if sil_score > best_score:
                best_score = sil_score
                best_db_score = db_score
                best_ch_score = ch_score
                best_params = params
    return best_params, best_score, best_db_score, best_ch_score

# Explanation for Step 9:
# The custom_grid_search function iterates through a set of parameters to identify the optimal configuration for the estimator.
# It evaluates clustering quality using three metrics:
# - Silhouette Score: Measures how similar each point is to its own cluster compared to other clusters, indicating cohesion and separation.
# - Davies-Bouldin Index: Evaluates the average similarity ratio between each cluster and the cluster that is most similar to it.
# - Calinski-Harabasz Index: Assesses the ratio of the sum of between-cluster dispersion to within-cluster dispersion.

# Step 10: Define the parameter grids
kmeans_param_grid = [
    {'n_clusters': n} for n in [3, 4, 5, 6, 7, 8, 9, 10]
]
dbscan_param_grid = [
    {'eps': eps, 'min_samples': min_samples} for eps in [0.3, 0.4, 0.5, 0.6, 0.7] for min_samples in [3, 5, 7, 10]
]
agglo_param_grid = [
    {'n_clusters': n_clusters, 'linkage': linkage} for n_clusters in [3, 4, 5, 6, 7, 8] for linkage in ['ward', 'complete', 'average', 'single']
]
spectral_param_grid = [
    {'n_clusters': n_clusters, 'affinity': affinity} for n_clusters in [3, 4, 5, 6, 7, 8] for affinity in ['nearest_neighbors', 'rbf']
]

# Explanation for Step 10:
# These parameter grids specify the range of values to be tested for each clustering algorithm.
# This allows us to systematically explore different configurations and identify the most effective setup for each algorithm.

# Step 11: Perform grid search for each clustering algorithm
best_kmeans_params, best_kmeans_sil_score, best_kmeans_db_score, best_kmeans_ch_score = custom_grid_search(KMeans(random_state=42), kmeans_param_grid, X_scaled)
best_dbscan_params, best_dbscan_sil_score, best_dbscan_db_score, best_dbscan_ch_score = custom_grid_search(DBSCAN(), dbscan_param_grid, X_scaled)
best_agglo_params, best_agglo_sil_score, best_agglo_db_score, best_agglo_ch_score = custom_grid_search(AgglomerativeClustering(), agglo_param_grid, X_scaled)
best_spectral_params, best_spectral_sil_score, best_spectral_db_score, best_spectral_ch_score = custom_grid_search(SpectralClustering(random_state=42), spectral_param_grid, X_scaled)

# Explanation for Step 11:
# This section performs the grid search for each clustering algorithm and evaluates them using the defined metrics.
# By comparing Silhouette, Davies-Bouldin, and Calinski-Harabasz scores, we can determine the best clustering algorithm and parameter configuration.
# This comprehensive evaluation helps ensure robust and meaningful clustering results for the Facebook friend graph data.

# Step 12: Print the best parameters and scores
print(f"Best KMeans parameters: {best_kmeans_params}, Silhouette Score: {best_kmeans_sil_score}, DB Score: {best_kmeans_db_score}, CH Score: {best_kmeans_ch_score}")
print(f"Best DBSCAN parameters: {best_dbscan_params}, Silhouette Score: {best_dbscan_sil_score}, DB Score: {best_dbscan_db_score}, CH Score: {best_dbscan_ch_score}")
print(f"Best Agglomerative Clustering parameters: {best_agglo_params}, Silhouette Score: {best_agglo_sil_score}, DB Score: {best_agglo_db_score}, CH Score: {best_agglo_ch_score}")
print(f"Best Spectral Clustering parameters: {best_spectral_params}, Silhouette Score: {best_spectral_sil_score}, DB Score: {best_spectral_db_score}, CH Score: {best_spectral_ch_score}")

# Explanation for Step 12:
# This cell prints the best parameters and corresponding scores for each clustering algorithm.
# The output provides a summary of the optimal configurations and their performance, guiding us towards the best approach for clustering the Facebook friend graph data.

# Step 13: Visualize the clustering results for the best algorithm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

def plot_clusters(X, labels, title):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=labels, palette='viridis')
    plt.title(title)
    plt.show()

# Use the best clustering results for visualization
best_labels = KMeans(n_clusters=best_kmeans_params['n_clusters'], random_state=42).fit_predict(X_scaled)
plot_clusters(X_scaled, best_labels, 'KMeans Clustering')

# Explanation for Step 13:
# This cell visualizes the clustering results using PCA for dimensionality reduction.
# The scatter plot shows the clusters in a 2D space, providing a clear visual representation of the clustering outcome.

# Step 14: Bar chart comparing the different algorithms' metrics
metrics_data = {
    'Algorithm': ['KMeans', 'DBSCAN', 'Agglomerative', 'Spectral'],
    'Silhouette Score': [best_kmeans_sil_score, best_dbscan_sil_score, best_agglo_sil_score, best_spectral_sil_score],
    'Davies-Bouldin Index': [best_kmeans_db_score, best_dbscan_db_score, best_agglo_db_score, best_spectral_db_score],
    'Calinski-Harabasz Index': [best_kmeans_ch_score, best_dbscan_ch_score, best_agglo_ch_score, best_spectral_ch_score]
}

metrics_df = pd.DataFrame(metrics_data)

# Plotting the bar chart
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
sns.barplot(x='Algorithm', y='Silhouette Score', data=metrics_df, ax=axes[0])
sns.barplot(x='Algorithm', y='Davies-Bouldin Index', data=metrics_df, ax=axes[1])
sns.barplot(x='Algorithm', y='Calinski-Harabasz Index', data=metrics_df, ax=axes[2])
axes[0].set_title('Silhouette Score')
axes[1].set_title('Davies-Bouldin Index')
axes[2].set_title('Calinski-Harabasz Index')
plt.tight_layout()
plt.show()

# Explanation for Step 14:
# This section creates a bar chart to compare the performance of the different clustering algorithms using the three metrics.
# The visual comparison helps in quickly identifying which algorithm performs the best across the different evaluation metrics.

# Due to runtime issues I was unable to produce outputs for this iteration of the model however the previous version with it's umap and degree of distribution visualizations are included.

# Based on the features identified in the previous iterations of the model and the goal of identfying major influencers with different constituents in the social media space I pivoted from using hop counts as advised by the rubric to Node2vec.
# Node2Vec embeddings inherently capture information about the neighborhood of each node, including nodes that are 2-3 hops away, as the random walks used in Node2Vec can traverse multiple hops.

# Also, from the previous iteration 3 distinct cluster of friends can be identified in the data.
# The degree of distribution plot also supports the few clusters shown in the umap as the right-skewed distribution tells us that there are a few influencers with a lot of friends but the majority of users are represented by low degree nodes with few connections.